In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# load in excel sheets
xls = pd.ExcelFile('Basketball_dataset.xlsx')
final_records = pd.read_excel(xls, 0)
game_records = pd.read_excel(xls, list(range(1,69)))

In [ ]:
# calculate score differential for all games
for i in range(1,69):
    game_records[i]['Score_diff'] = game_records[i]['Tm'] - game_records[i]['Opp']
    game_records[i]['Score_diff_norm'] = game_records[i]['Score_diff'] / (game_records[i]['Tm'] + game_records[i]['Opp']) 
    game_records[i]['Score_diff_ratio'] = game_records[i]['Tm'] / game_records[i]['Opp']
    game_records[i]['Score_diff_ratio'] = np.where(game_records[i]['Score_diff_ratio'] > 1, game_records[i]['Score_diff_ratio'], np.reciprocal(game_records[i]['Score_diff_ratio']) )



In [ ]:
print(game_records[1]['Score_diff_ratio'])

In [ ]:
# calculate the total score diff
final_records["Score_diff"] = np.NaN
final_records["Score_diff_norm"] = np.NaN
for i in range(1,69):    
    final_records["Score_diff"][i-1] = np.sum(game_records[i]['Score_diff'])
    final_records["Score_diff_norm"][i-1] = np.sum(game_records[i]['Score_diff_norm'])



In [ ]:
# print output
print(final_records)
#print(game_records[1]['Score_diff'])

In [ ]:
plt.plot(final_records["Seed"],final_records["Score_diff"])
plt.plot(final_records["Seed"],150*final_records["Score_diff_norm"])
plt.xlabel("Team Playoff Seed")
plt.ylabel("Team Season Score differential")
plt.title("Playoff Seed vs Score differential")

In [ ]:

plt.plot(final_records["Seed"],final_records["Score_diff_norm"])
plt.xlabel("Team Playoff Seed")
plt.ylabel("Team Season Score differential")
plt.title("Playoff Seed vs Score differential")